In [1]:
import json
import logging
from sierra_ils_utils import SierraRESTAPI, RecordDateRange

# Configure the root logger
logging.basicConfig(level=logging.DEBUG)

# Load configuration
with open('./.config.json') as f:
    config = json.load(f)

# Create and configure the logger for the current module
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# Optionally, set the logging level for the sierra_ils_utils module
logging.getLogger('sierra_ils_utils').setLevel(logging.DEBUG)

# Initialize SierraRESTAPI with DEBUG logging level
sierra_api = SierraRESTAPI(
    sierra_api_base_url=config.get('sierra_api_base_url'),
    sierra_api_key=config.get('sierra_api_key'),
    sierra_api_secret=config.get('sierra_api_secret'),
    log_level=logging.DEBUG
)


In [4]:
update_date = RecordDateRange(start_date=-1) 

In [5]:
str(update_date)

'[1969-12-31T18:59:59Z,]'

In [6]:
update_date = RecordDateRange(exact_date='2021-12-21')
print(str(update_date))

try:
    update_date.advance_range('minutes=5')
except TypeError as e:
    print(e)  # this should fail
    error = e

2021-12-21
start_date and end_date must be datetime objects


In [7]:
e

NameError: name 'e' is not defined

In [3]:
update_date = RecordDateRange(
    start_date='2021-12-21T00:00:00',
    end_date='2021-12-22T00:00:00'
)
print(str(update_date))

try:
    update_date.advance_range('minutes=5')  # advance the interval by 5 minutes
    print(str(update_date))
except TypeError as e:
    print(e)  # this should NOT fail

try:
    update_date.advance_range('minutes=5')  # advance the interval by 5 minutes
    print(str(update_date))
except TypeError as e:
    print(e)  # this should NOT fail


[2021-12-21T00:00:00Z,2021-12-22T00:00:00Z]
[2021-12-22T00:00:01Z,2021-12-22T00:05:00Z]
[2021-12-22T00:05:01Z,2021-12-22T00:10:00Z]


In [3]:
"""
Range syntax applies to dates and record IDs.

Range syntax has the form:

[<startValue>,<endValue>]

The start and end values are inclusive.

You can specify ranges in the following ways:

Exactly (a search parameter)

updatedDate=2013-12-10T17:16:35Z

id=1000004

From <startValue> to <endValue> (inclusive)

updatedDate=[2013-12-10T17:16:35Z,2013-12-13T21:34:35Z]

id=[1000004,1000054]

<startValue> and after (inclusive)

updatedDate=[2013-12-10T17:16:35Z,]

id=[1000004,]

Up to and including <endValue>

updatedDate=[,2013-12-13T21:34:35Z]

id=[,1000054]

Dates must match the date format of the property to be retrieved. In most cases, the format is ISO 8601 combined date and time in UTC with Z (zero) offset. Some date properties, such as catalogDate and deletedDate, are date only, with no time. Refer to the bib object and item object descriptions and examples for more information.
"""


2021-12-21T00:00:00Z


In [3]:
update_date = RecordDateRange(exact_date='2021-12-21')
print(str(update_date))

[,]


In [2]:
from datetime import datetime, timedelta

start_date = (
    datetime.now() - timedelta(hours=1)  # get the current timestamp and subtract an hour
).isoformat()                            # format as iso

In [3]:
start_date

'2023-11-27T13:31:20.907282'

In [6]:
from datetime import datetime, timedelta

start_date = (
    datetime.now() - timedelta(hours=1)  # get the current timestamp and subtract an hour
).isoformat()                            # format as iso

end_date=datetime.now().isoformat()

update_date_range = RecordDateRange(
    start_date=start_date,
    end_date=datetime.now().isoformat()
    # start_date='2023-11-27T00:00:00', 
    # end_date='2023-11-28T00:00:00'
)

params = {
    'updatedDate': str(update_date_range),  # the param needs to be formatted as a string
    'limit': 2000,
}

results = sierra_api.get(
    'bibs/',
    params=params
)

INFO:sierra_ils_utils.sierra_ils_utils:GET {"endpoint": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/", "params": "{'updatedDate': '[2023-11-27T13:33:26.624254Z,2023-11-27T14:33:26.624334Z]', 'limit': 2000}"}
DEBUG:urllib3.connectionpool:Resetting dropped connection: classic.cincinnatilibrary.org
DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "GET /iii/sierra-api/v6/bibs/?updatedDate=%5B2023-11-27T13%3A33%3A26.624254Z%2C2023-11-27T14%3A33%3A26.624334Z%5D&limit=2000 HTTP/1.1" 200 None
INFO:sierra_ils_utils.sierra_ils_utils:GET https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/?updatedDate=%5B2023-11-27T13%3A33%3A26.624254Z%2C2023-11-27T14%3A33%3A26.624334Z%5D&limit=2000 200 ✅


In [10]:
results.data.total

1188

In [3]:
date_range = RecordDateRange(
    start_date='2023-11-01T00:00:00',
    end_date='2023-11-01T01:00:00'
)
print("Initial Range (String):", date_range)

# Advance the range by 5 minutes
date_range.advance_range('minutes=5')
print("Advanced Range (String):", date_range)

# Advance the range by 5 minutes
date_range.advance_range('minutes=5')
print("Advanced Range (String):", date_range)

Initial Range (String): [2023-11-01T00:00:00Z,2023-11-01T01:00:00Z]
Advanced Range (String): [2023-11-01T01:00:01Z,2023-11-01T01:05:00Z]
Advanced Range (String): [2023-11-01T01:05:01Z,2023-11-01T01:10:00Z]


In [7]:
# Advance the range by 5 minutes
date_range.advance_range('minutes=5')
print("Advanced Range (String):", date_range)

Advanced Range (String): [2023-11-01T01:05:01Z,2023-11-01T01:10:00Z]


In [9]:
# should advance the range by another 5 minutes
date_range.advance_range('minutes=5')
print("Advanced Range (String):", date_range)

Advanced Range (String): [2023-11-01T01:10:01Z,2023-11-01T01:15:00Z]


In [14]:
date_range_string = RecordDateRange(
    start_date='2023-11-01',
    end_date='2023-11-02'
)

print(date_range_string)

[2023-11-01T00:00:00Z,2023-11-02T00:00:00Z]


In [27]:
date_range = RecordDateRange(
    start_date='2023-11-01',
    end_date='2023-11-02'
)

# print(date_range_string)
print(date_range)

[2023-11-01T00:00:00Z,2023-11-02T00:00:00Z]


In [30]:
date_range.advance_range('minutes=5')

In [31]:
print(date_range)

[2023-11-01T00:10:00Z,2023-11-02T00:10:00Z]


In [25]:
print(update)

update.advance_range('minutes=5')

2023-11-01T00:00:00Z
